In [2]:
# 住宿資訊
from requests import get
from bs4 import BeautifulSoup
from time import sleep             #不能狂爬,要用 sleep 設停頓避免被反爬蟲禁IP
import openpyxl

wb = openpyxl.Workbook()
ws = wb.active

dic1 = {}
info_all = []
url_base = "https://hiking.biji.co"
page = 0
url_rent = "https://hiking.biji.co/index.php?q=place&type=1&page="
data=["init"]
while len(data) != 0:
    page += 1
    res = get(url_rent+str(page))
    soup = BeautifulSoup(res.text)
    data = soup.find_all('div', class_="flex-1 p-5 space-y-2.5")
    for i in range(len(data)):
        url = url_base + data[i].h2.a["href"]
        sub_res = get(url)
        sub_soup = BeautifulSoup(sub_res.text)
        sub_data_tag =sub_soup.find('dl', class_="border").find_all('dt')
        sub_data_value =sub_soup.find('dl', class_="border").find_all('dd')
        info = [""]*100
        for j in range(len(sub_data_tag)):
            dic1[sub_data_tag[j].text] = dic1.get(sub_data_tag[j].text,len(dic1))
            if sub_data_tag[j].text == "營業時間":
                info[dic1["備註"]] = sub_data_value[j].text
            elif sub_data_value[j].find("a") == None:
                info[dic1[sub_data_tag[j].text]] = sub_data_value[j].text
            else:
                info[dic1[sub_data_tag[j].text]] = url_base + sub_data_value[j].find("a")["href"]
        info_all.append(info)
ws.append(list(dic1.keys()))
for i in range(len(info_all)):
    ws.append(info_all[i])
wb.save('健行筆記住宿.xlsx')